In [1]:
import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon
import scanpy as sc

import time


In [ ]:
from scipy.sparse import issparse
import muon.prot as pt



for path in ['../../datasets/Human_Lymph_Node_A1/','../../datasets/Human_Lymph_Node_D1/']:

    adata_omics1 = sc.read_h5ad(path + 'adata_RNA.h5ad')
    adata_omics2 = sc.read_h5ad(path + 'adata_ADT.h5ad')
       
    adata_omics1.var_names_make_unique()
    adata_omics2.var_names_make_unique()
    
    adata_omics1.obs['batch'] = 'Thymus'
    adata_omics1.layers['counts'] = adata_omics1.X.copy()
    
    sc.pp.filter_genes(adata_omics2, min_cells=10)
    sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
    sc.pp.normalize_total(adata_omics1, target_sum=1e4)
    sc.pp.log1p(adata_omics1)
      
    adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
    sc.pp.pca(adata_omics1_high)
      
    # Protein
    pt.pp.clr(adata_omics2)
    sc.pp.scale(adata_omics2)
    sc.pp.pca(adata_omics2)
    
    
    mdata = md.MuData({'rna': adata_omics1, 'adt': adata_omics2})
    
    sc.pp.neighbors(mdata.mod["rna"],n_neighbors=10)
    sc.pp.neighbors(mdata.mod["adt"],n_neighbors=10)
    muon.pp.neighbors(mdata,n_neighbors=10)

    muon.tl.umap(mdata)
    
    adata=sc.AnnData(X=mdata.X,obs=mdata.obs,uns=mdata.uns,obsm=mdata.obsm,obsp=mdata.obsp)
    adata.obsm.update(adata_omics1.obsm)

    result_path=path.replace("datasets","results")
    adata.write_h5ad(result_path+'/adata_WNN.h5ad')

/home/ws6tg/anaconda3/envs/scvi/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:325: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm["X_pca"] = X_pca
/home/ws6tg/anaconda3/envs/scvi/lib/python3.9/site-packages/scanpy/tools/_utils.py:41: UserWarning: You’re trying to run this on 23375 dimensions of `.X`, if you really want this, set `use_rep='X'`.
         Falling back to preprocessing with `sc.pp.pca` and default params.
  warnings.warn(
